# <center>  Cousera Capstone Project
# <center> IBM Applied Data Science Capstone
#### <center> Ajitesh Kohli    
#### <center> May 2, 2020   

### <center>  _Exploring a place where opening a Shopping Complex would be more beneficial in Delhi NCR, India_ </center>


### Importing Libraries

In [1]:
%%capture
import numpy as np
import pandas as pd
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install html5lib
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes 
import folium


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


In [3]:
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="delhi_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Delhi, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Delhi, India are 28.6141793, 77.2022662.


In [4]:
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)
map_delhi

## Defining Foursquare credentials and version

In [5]:
CLIENT_ID = 'PBFP5HJT00LXB0QRPXGDTZQI0PO2MFMG4IFQ3K1OKH0WPKPX' # your Foursquare ID
CLIENT_SECRET = '5VVO420NBKH1FDD2AWTYUUQIWUF5IM3LQBHKNRRDHQFRCNY4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PBFP5HJT00LXB0QRPXGDTZQI0PO2MFMG4IFQ3K1OKH0WPKPX
CLIENT_SECRET:5VVO420NBKH1FDD2AWTYUUQIWUF5IM3LQBHKNRRDHQFRCNY4


### Now, let's get the top 100 venues that are within a radius of 50 kilometers.

In [6]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    


In [7]:
radius = 50000
LIMIT = 100

   


offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng','city'])

while (True):
    
         
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,
                                                                                                                                             latitude,longitude,radius,LIMIT,offset)
        
        result = requests.get(url).json()
        venues_fetched = len(result['response']['groups'][0]['items'])
        total_venues = total_venues + venues_fetched
        print("Total {} venues fetched within a total radius of {} Km".format(venues_fetched, radius/1000))

        venues = result['response']['groups'][0]['items']
        venues = json_normalize(venues)

        # Filter the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city']
        venues = venues.loc[:, filtered_columns]

         # Filter the category for each row
        venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

        # Clean all column names
        venues.columns = [col.split(".")[-1] for col in venues.columns]
        foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
        if (venues_fetched < 50):
            break
        else:
            offset = offset + 20

foursquare_venues = foursquare_venues.reset_index(drop = True)
print("\nTotal {} venues fetched".format(total_venues))


Total 100 venues fetched within a total radius of 50.0 Km


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Total 100 venues fetched within a total radius of 50.0 Km
Total 100 venues fetched within a total radius of 50.0 Km
Total 100 venues fetched within a total radius of 50.0 Km
Total 98 venues fetched within a total radius of 50.0 Km
Total 78 venues fetched within a total radius of 50.0 Km
Total 58 venues fetched within a total radius of 50.0 Km
Total 38 venues fetched within a total radius of 50.0 Km

Total 672 venues fetched


In [8]:
#venues = result['response']['groups'][0]['items']
#venues = json_normalize(venues)
#venues

In [9]:
foursquare_venues.shape

(672, 5)

In [10]:
four_df=foursquare_venues
#foursquare_venues=four_df

#### Checking for the duplicate values in data as the fetching limit was only 100 and I tried to fetch it multiple times by changing offset 

In [11]:
foursquare_venues.drop_duplicates(subset=None, keep='first', inplace=True)
foursquare_venues.shape

(178, 5)

In [12]:
foursquare_venues.head(20)

,name,categories,lat,lng,city
0,The Imperial,Hotel,28.625548,77.218664,New Delhi
1,Tamra,Restaurant,28.620543,77.218174,New Delhi
2,Pandey Paan,Smoke Shop,28.622249,77.201075,New Delhi
3,The Big Chill Cafe,Italian Restaurant,28.600686,77.227636,New Delhi
4,Amour Bistro,Café,28.601569,77.185923,NaN
5,The Leela Palace,Hotel,28.579923,77.189291,New Delhi
6,Varq | वर्क,Indian Restaurant,28.604547,77.223781,New Delhi
7,Naturals Ice Cream,Ice Cream Shop,28.634455,77.222139,New Delhi
8,The Lodhi Hotel,Hotel,28.591669,77.238131,New Delhi
9,Khan Market | खान मार्केट (Khan Market),Market,28.600342,77.226923,New Delhi


In [13]:
foursquare_venues.dropna(subset=["city"],axis=0,inplace=True)
foursquare_venues.head()
                               

,name,categories,lat,lng,city
0,The Imperial,Hotel,28.625548,77.218664,New Delhi
1,Tamra,Restaurant,28.620543,77.218174,New Delhi
2,Pandey Paan,Smoke Shop,28.622249,77.201075,New Delhi
3,The Big Chill Cafe,Italian Restaurant,28.600686,77.227636,New Delhi
5,The Leela Palace,Hotel,28.579923,77.189291,New Delhi


In [14]:
# Map of Delhi already createdusing latitude and longitude values


# add markers to map
for lat, lng, name, categories in zip(foursquare_venues['lat'],foursquare_venues['lng'],foursquare_venues['name'],foursquare_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],radius=2,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi

In [15]:
df_one=foursquare_venues
#foursquare_venues=df_one

In [16]:
foursquare_venues

,name,categories,lat,lng,city
0,The Imperial,Hotel,28.625548,77.218664,New Delhi
1,Tamra,Restaurant,28.620543,77.218174,New Delhi
2,Pandey Paan,Smoke Shop,28.622249,77.201075,New Delhi
3,The Big Chill Cafe,Italian Restaurant,28.600686,77.227636,New Delhi
5,The Leela Palace,Hotel,28.579923,77.189291,New Delhi
6,Varq | वर्क,Indian Restaurant,28.604547,77.223781,New Delhi
7,Naturals Ice Cream,Ice Cream Shop,28.634455,77.222139,New Delhi
8,The Lodhi Hotel,Hotel,28.591669,77.238131,New Delhi
9,Khan Market | खान मार्केट (Khan Market),Market,28.600342,77.226923,New Delhi
10,Lodhi Gardens (लोधी बाग़) (Lodhi Gardens),Park,28.591424,77.220899,New Delhi


In [17]:
foursquare_venues.groupby(['city','categories']).count()

name  lat  lng
city          categories                              
Delhi         Café                         1    1    1
              Hotel                        1    1    1
              Indian Restaurant            2    2    2
              Italian Restaurant           1    1    1
              Lounge                       1    1    1
              Restaurant                   1    1    1
              Speakeasy                    1    1    1
Farīdābād     Café                         3    3    3
              Deli / Bodega                1    1    1
              Fried Chicken Joint          1    1    1
              Hotel                        1    1    1
              Indian Restaurant            1    1    1
              Multiplex                    1    1    1
              Pizza Place                  3    3    3
              Restaurant                   1    1    1
              Sandwich Place               1    1    1
Ghāziābād     Food Court                   1    1    1
Greater Noida Fast Food Restaurant         3    3    3
              Hotel                        2    2    2
              Indian Sweet Shop            1    1    1
              Monument / Landmark          1    1    1
              Multiplex                    1    1    1
              Racetrack                    1    1    1
              Shopping Mall                2    2    2
Gurgaon       Bagel Shop                   1    1    1
              Bakery                       1    1    1
              Bowling Alley                1    1    1
              Brewery                      2    2    2
              Café                         5    5    5
              Clothing Store               1    1    1
              Coffee Shop                  2    2    2
              Falafel Restaurant           1    1    1
              Fast Food Restaurant         2    2    2
              General Entertainment        1    1    1
              Golf Course                  1    1    1
              Hotel                        5    5    5
              Ice Cream Shop               1    1    1
              Indian Restaurant            7    7    7
              Market                       2    2    2
              Moroccan Restaurant          1    1    1
              Park                         1    1    1
              Pizza Place                  3    3    3
              Plaza                        1    1    1
              Resort                       1    1    1
              Restaurant                   1    1    1
              Sandwich Place               3    3    3
              Shopping Mall                2    2    2
              Supermarket                  1    1    1
Mahrauli      Thai Restaurant              1    1    1
Meerut        Juice Bar                    1    1    1
Modinagar     Juice Bar                    1    1    1
Murthal       Indian Restaurant            1    1    1
NCR           Hotel                        1    1    1
New Delhi     Art Gallery                  1    1    1
              Art Museum                   1    1    1
              Asian Restaurant             1    1    1
              Bakery                       1    1    1
              Bar                          3    3    3
              Boutique                     1    1    1
              Café                         3    3    3
              Coffee Shop                  1    1    1
              Department Store             1    1    1
              Gastropub                    1    1    1
              Golf Course                  1    1    1
              Gym / Fitness Center         1    1    1
              Historic Site                1    1    1
              Hotel                        8    8    8
              Ice Cream Shop               1    1    1
              Indian Restaurant            7    7    7
              Irani Cafe                   1    1    1
              Italian Restaurant           4    4    4
              Lounge     

In [18]:
foursquare_venues.replace('Mahrauli','Gurgaon',inplace=True)

#### Let's group the data by city and categories

In [19]:
foursquare_venues.groupby(['city','categories']).count()

name  lat  lng
city          categories                              
Delhi         Café                         1    1    1
              Hotel                        1    1    1
              Indian Restaurant            2    2    2
              Italian Restaurant           1    1    1
              Lounge                       1    1    1
              Restaurant                   1    1    1
              Speakeasy                    1    1    1
Farīdābād     Café                         3    3    3
              Deli / Bodega                1    1    1
              Fried Chicken Joint          1    1    1
              Hotel                        1    1    1
              Indian Restaurant            1    1    1
              Multiplex                    1    1    1
              Pizza Place                  3    3    3
              Restaurant                   1    1    1
              Sandwich Place               1    1    1
Ghāziābād     Food Court                   1    1    1
Greater Noida Fast Food Restaurant         3    3    3
              Hotel                        2    2    2
              Indian Sweet Shop            1    1    1
              Monument / Landmark          1    1    1
              Multiplex                    1    1    1
              Racetrack                    1    1    1
              Shopping Mall                2    2    2
Gurgaon       Bagel Shop                   1    1    1
              Bakery                       1    1    1
              Bowling Alley                1    1    1
              Brewery                      2    2    2
              Café                         5    5    5
              Clothing Store               1    1    1
              Coffee Shop                  2    2    2
              Falafel Restaurant           1    1    1
              Fast Food Restaurant         2    2    2
              General Entertainment        1    1    1
              Golf Course                  1    1    1
              Hotel                        5    5    5
              Ice Cream Shop               1    1    1
              Indian Restaurant            7    7    7
              Market                       2    2    2
              Moroccan Restaurant          1    1    1
              Park                         1    1    1
              Pizza Place                  3    3    3
              Plaza                        1    1    1
              Resort                       1    1    1
              Restaurant                   1    1    1
              Sandwich Place               3    3    3
              Shopping Mall                2    2    2
              Supermarket                  1    1    1
              Thai Restaurant              1    1    1
Meerut        Juice Bar                    1    1    1
Modinagar     Juice Bar                    1    1    1
Murthal       Indian Restaurant            1    1    1
NCR           Hotel                        1    1    1
New Delhi     Art Gallery                  1    1    1
              Art Museum                   1    1    1
              Asian Restaurant             1    1    1
              Bakery                       1    1    1
              Bar                          3    3    3
              Boutique                     1    1    1
              Café                         3    3    3
              Coffee Shop                  1    1    1
              Department Store             1    1    1
              Gastropub                    1    1    1
              Golf Course                  1    1    1
              Gym / Fitness Center         1    1    1
              Historic Site                1    1    1
              Hotel                        8    8    8
              Ice Cream Shop               1    1    1
              Indian Restaurant            7    7    7
              Irani Cafe                   1    1    1
              Italian Restaurant           4    4    4
              Lounge     

# Analysing the venues 

In [20]:
# one hot encoding
df_onehot = pd.get_dummies(foursquare_venues[['categories']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe

df_onehot['city'] = foursquare_venues[['city']]

# move postal, borough and neighborhood column to the first column
fixed_columns = list(df_onehot.columns[-1:]) + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head(10)

(150, 54)


,city,Art Gallery,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bar,Boutique,Bowling Alley,Brewery,Café,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Falafel Restaurant,Fast Food Restaurant,Food Court,Fried Chicken Joint,Gastropub,General Entertainment,Golf Course,Gym / Fitness Center,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Irani Cafe,Italian Restaurant,Juice Bar,Lounge,Market,Mediterranean Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Pizza Place,Plaza,Pub,Racetrack,Resort,Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,South Indian Restaurant,Speakeasy,Stadium,Supermarket,Thai Restaurant
0,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,New Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###  Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [21]:
df_grouped = df_onehot.groupby(["city"]).mean().reset_index()

print(df_grouped.shape)
df_grouped

(14, 54)


,city,Art Gallery,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bar,Boutique,Bowling Alley,Brewery,Café,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Falafel Restaurant,Fast Food Restaurant,Food Court,Fried Chicken Joint,Gastropub,General Entertainment,Golf Course,Gym / Fitness Center,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Irani Cafe,Italian Restaurant,Juice Bar,Lounge,Market,Mediterranean Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Pizza Place,Plaza,Pub,Racetrack,Resort,Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,South Indian Restaurant,Speakeasy,Stadium,Supermarket,Thai Restaurant
0,Delhi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.250000,0.000000,0.000000,0.125000,0.0,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000
1,Farīdābād,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.230769,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.0,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.230769,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,Ghāziābād,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
3,Greater Noida,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.272727,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.090909,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.090909,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
4,Gurgaon,0.000000,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000,0.020833,0.041667,0.104167,0.020833,0.041667,0.000000,0.000000,0.020833,0.041667,0.0,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000,0.104167,0.020833,0.145833,0.000000,0.000000,0.000000,0.0,0.000000,0.041667,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.020833,0.062500,0.020833,0.0,0.000000,0.020833,0.020833,0.062500,0.041667,0.000000,0.000000,0.000,0.000000,0.020833,0.020833
5,Meerut,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
6,Modinagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,Murth

###  Let's write a function to sort the venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each City.

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_sorted = pd.DataFrame(columns=columns)
df_sorted['city'] = df_grouped['city']

for ind in np.arange(df_grouped.shape[0]):
    df_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

df_sorted.head()

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Delhi,Indian Restaurant,Speakeasy,Italian Restaurant,Restaurant,Lounge,Café,Hotel,Thai Restaurant,Falafel Restaurant,Gym / Fitness Center
1,Farīdābād,Pizza Place,Café,Fried Chicken Joint,Sandwich Place,Multiplex,Hotel,Restaurant,Deli / Bodega,Indian Restaurant,Gastropub
2,Ghāziābād,Food Court,Thai Restaurant,Indian Restaurant,Hotel,Historic Site,Gym / Fitness Center,Golf Course,General Entertainment,Gastropub,Fried Chicken Joint
3,Greater Noida,Fast Food Restaurant,Shopping Mall,Hotel,Indian Sweet Shop,Multiplex,Racetrack,Monument / Landmark,Gym / Fitness Center,Golf Course,General Entertainment
4,Gurgaon,Indian Restaurant,Café,Hotel,Sandwich Place,Pizza Place,Brewery,Shopping Mall,Fast Food Restaurant,Market,Coffee Shop


# Cluster Neighborhoods

#### Run *k*-means to cluster the neighborhood into 5 clusters.

In [49]:
# set number of clusters
kclusters = 9

df_grouped_clustering = df_grouped.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 8, 0, 7, 1, 2, 2, 3, 4, 1, 5, 3, 4, 6], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each city

In [50]:
# add clustering labels
df_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [51]:
df_merged = foursquare_venues

# merge df_grouped with df_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(df_sorted.set_index('city'), on='city')

df_merged.head() # check the last columns!

,name,categories,lat,lng,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Imperial,Hotel,28.625548,77.218664,New Delhi,1,Hotel,Indian Restaurant,Italian Restaurant,Shopping Mall,Café,Bar,Market,Restaurant,Monument / Landmark,Historic Site
1,Tamra,Restaurant,28.620543,77.218174,New Delhi,1,Hotel,Indian Restaurant,Italian Restaurant,Shopping Mall,Café,Bar,Market,Restaurant,Monument / Landmark,Historic Site
2,Pandey Paan,Smoke Shop,28.622249,77.201075,New Delhi,1,Hotel,Indian Restaurant,Italian Restaurant,Shopping Mall,Café,Bar,Market,Restaurant,Monument / Landmark,Historic Site
3,The Big Chill Cafe,Italian Restaurant,28.600686,77.227636,New Delhi,1,Hotel,Indian Restaurant,Italian Restaurant,Shopping Mall,Café,Bar,Market,Restaurant,Monument / Landmark,Historic Site
5,The Leela Palace,Hotel,28.579923,77.189291,New Delhi,1,Hotel,Indian Restaurant,Italian Restaurant,Shopping Mall,Café,Bar,Market,Restaurant,Monument / Landmark,Historic Site


### Now let's visualise the resulting clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['lat'], df_merged['lng'],df_merged['name'],df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## **Conclusion**

##So for a business establishment according to the data in hand, it would be advisable to open a new mall in either Delhi region or Gurgaon region. 
